In [1]:
import torch

In [2]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [3]:
x.requires_grad_(True)
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [4]:
x.grad

In [5]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [6]:
y = 2 * x@x  # 2 * torch.dot(x, x)랑 똑같음
y

tensor(28., grad_fn=<DotBackward0>)

In [7]:
y.backward()

In [8]:
x.grad  # 4x 나와야 함

tensor([ 0.,  4.,  8., 12.])

In [9]:
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [10]:
# J_ij = y_i를 x_j로 미분한 값
# PyTorch에서는 이걸 그대로 계산하는게 아님
# v^T와 J를 곱한 값을 계산함
# 이 v를 gradient라 함
# x: (n)
# y: (m)
# J: (m, n)
# v: (m)

In [11]:
x.grad.zero_()

tensor([0., 0., 0., 0.])

In [12]:
y = x * x  # elementwise
y

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)

In [13]:
# y = (x_1**2, x_2**2, x_3**2, x_4**2)
# J_ij = y_i를 x_j로 미분한 값 = 2*x_j
# J = 2 * [x_1 0 0 0]
#         [0 x_2 0 0] 
#         [0 0 x_3 0] 
#         [0 0 0 x_4] 
# v = [1, 1, 1, 1]^T
# v^T@J = 2*[x_1, x_2, x_3, x_4]

In [14]:
v = torch.ones(len(y))
y.backward(gradient=v)
x.grad

tensor([0., 2., 4., 6.])

In [15]:
x.grad.zero_()
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [16]:
y = x * x
y

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)

In [17]:
u = y.detach()
u

tensor([0., 1., 4., 9.])

In [18]:
z = u * x
z

tensor([ 0.,  1.,  8., 27.], grad_fn=<MulBackward0>)

In [19]:
z.sum().backward()  # 오로지 x에 대해서만 계산됨
                    # z = u *  (z 입장에서 u는 상수이고 x만 변수임)

In [20]:
u == x.grad

tensor([True, True, True, True])

In [21]:
x.grad.zero_()
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [22]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [23]:
a = torch.randn(size=(), requires_grad=True)  # N(0, 1)에서 랜덤하게 한 개 뽑아라
print(a)                                      # requires_grad=True니까 a로 뭔가를 미분할 수 있음

d = f(a)
d.backward()  # d를 a로 미분해라
a.grad        # d를 a로 미분한 값이다

tensor(-0.1213, requires_grad=True)


tensor(1638400.)

https://zhang-yang.medium.com/the-gradient-argument-in-pytorchs-backward-function-explained-by-examples-68f266950c29

In [24]:
x = torch.tensor(1., requires_grad=True)  # x로 뭔가를 미분할 것이다
x

tensor(1., requires_grad=True)

In [25]:
y = x**2
y

tensor(1., grad_fn=<PowBackward0>)

In [26]:
y.backward()  # gradient에 1 넣은거랑 똑같음

In [27]:
x.grad  # 2x가 나와야 함

tensor(2.)

In [28]:
x = torch.tensor(1., requires_grad=True)  # x로 뭔가를 미분할 것이다
y = x**2
y.backward(gradient=torch.tensor(100.))

In [29]:
x.grad

tensor(200.)

In [30]:
x = torch.tensor([1., 2.], requires_grad=True)  # x로 뭔가를 미분할 것이다
y = sum(x)
y.backward()
x.grad  # [1, 1] 나와야 함

tensor([1., 1.])

In [31]:
x = torch.tensor([1., 2.], requires_grad=True)  # x로 뭔가를 미분할 것이다
y = sum(x)
y.backward(gradient=torch.tensor(100.))
x.grad  # [100, 100] 나와야 함

tensor([100., 100.])

In [32]:
x = torch.tensor([1., 2.], requires_grad=True)  # x로 뭔가를 미분할 것이다
y = 3*x**2
# Jacobian
# y = [3*x1**2, 3*x2**2]
# J_ij = yi를 xj로 미분한 값
# J: [[6*x_1, 0, 0, 6*x_2]]
y.backward(torch.tensor([1., 1.]))
x.grad  # [6, 12] 나와야 함

tensor([ 6., 12.])